<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_de_aloca%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     |████████████████████████████████| 9.0MB 2.8MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 256kB 43.4MB/s 
     |████████████████████████████████| 163kB 47.1MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-

In [ ]:
## Modelo matemático ##
## Min x11 + 4x12 + 6x13 + 3x14 + 9x21 + 7x22 + 10x23 + 9x24 + 4x31 + 5x32 + 11x33 + 7x34 +8x41 + 7x42 + 8x43 + 5x44 ##
##                               ##
## S.a x11 + x12 + x13 + x14 = 1 ##
##     x21 + x22 + x23 + x24 = 1 ##
##     x31 + x32 + x33 + x34 = 1 ##
##     x41 + x42 + x43 + x44 = 1 ##
##                               ##
##     x11 + x21 + x31 + x41 = 1 ##
##     x12 + x22 + x32 + x42 = 1 ##
##     x13 + x23 + x33 + x43 = 1 ##
##     x14 + x24 + x34 + x44 = 1 ##


## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv

## Dados do problema ##
custos = [[1, 4, 6, 3], [9, 7, 10, 9], [4, 5, 11, 7], [8, 7, 8, 5]]
m = len(custos[0])
n = len(custos[0])

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices = pyEnv.RangeSet(m)
modelo.Indices2 = pyEnv.RangeSet(n)

## Criando as variáveis ##
modelo.Variaveis = pyEnv.Var(modelo.Indices, modelo.Indices2, within = pyEnv.NonNegativeReals)
modelo.Custo_alocacao = pyEnv.Param(modelo.Indices, modelo.Indices2, initialize = lambda modelo, i, j: custos[i-1][j-1])

## Criando a Função Objetivo ##
def Objetivo(modelo):
  return sum(modelo.Variaveis[i,j] * modelo.Custo_alocacao[i,j] for i in modelo.Indices for j in modelo.Indices2)

modelo.Objetivo = pyEnv.Objective(rule = Objetivo, sense = pyEnv.minimize)

## Criando as restrições ##
def rest1(modelo, i):
  return sum(modelo.Variaveis[i,j] for j in modelo.Indices2) == 1

def rest2(modelo, j):
  return sum(modelo.Variaveis[i,j] for i in modelo.Indices) == 1

modelo.rest1 = pyEnv.Constraint(modelo.Indices, rule = rest1)
modelo.rest2 = pyEnv.Constraint(modelo.Indices2, rule = rest2)

## Chamando o Solver ##
solver = pyEnv.SolverFactory('glpk', executable = '/usr/bin/glpsol')
result_objetivo = solver.solve(modelo, tee = True)

## Printando o resultado ##
lista = list(modelo.Variaveis.keys())
print()
print()
print()
modelo.Objetivo()
print(result_objetivo)
print()
for i in lista:
  if modelo.Variaveis[i]() != 0:
    print(i, '---', modelo.Variaveis[i]())
print()
print('Valor da função objetivo =', modelo.Objetivo())

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpgxz77lls.glpk.raw --wglp /tmp/tmp_qt4c1_8.glpk.glp --cpxlp
 /tmp/tmpf1r35453.pyomo.lp
Reading problem data from '/tmp/tmpf1r35453.pyomo.lp'...
9 rows, 17 columns, 33 non-zeros
100 lines were read
Writing problem data to '/tmp/tmp_qt4c1_8.glpk.glp'...
87 lines were written
GLPK Simplex Optimizer, v4.65
9 rows, 17 columns, 33 non-zeros
Preprocessing...
8 rows, 16 columns, 32 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
      0: obj =   2.000000000e+01 inf =   1.000e+00 (1)
      2: obj =   2.400000000e+01 inf =   0.000e+00 (0)
*     4: obj =   2.100000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (42457 bytes)
Writing basic solution to '/tmp/tmpgxz77lls.glpk.raw'...
35 lines were written




Problem: 
- Na